In [1]:
import pandas as pd
import httpx
import json
from bs4 import BeautifulSoup

# Data Extraction

In [2]:
acting_categories = (
    "outstanding-lead-actor-in-a-comedy-series",
    "outstanding-supporting-actor-in-a-comedy-series",
    "outstanding-guest-actor-in-a-comedy-series",

    "outstanding-lead-actor-in-a-drama-series",
    "outstanding-supporting-actor-in-a-drama-series",
    "outstanding-guest-actor-in-a-drama-series",

    "outstanding-lead-actor-in-a-miniseries-or-a-movie",
    "outstanding-supporting-actor-in-a-miniseries-or-a-movie",
    
    "outstanding-lead-actress-in-a-comedy-series",
    "outstanding-supporting-actress-in-a-comedy-series",
    "outstanding-guest-actress-in-a-comedy-series",

    "outstanding-lead-actress-in-a-drama-series",
    "outstanding-supporting-actress-in-a-drama-series",
    "outstanding-guest-actress-in-a-drama-series",

    "outstanding-lead-actress-in-a-miniseries-or-a-movie",
    "outstanding-supporting-actress-in-a-miniseries-or-a-movie",
)

In [3]:
show_categories = (
    "outstanding-comedy-series",
    "outstanding-drama-series",
    "outstanding-miniseries",
    "outstanding-made-for-television-movie",
)

In [4]:
def get_emmys_page(year: int, category: str):
    url = f"https://www.emmys.com/awards/nominees-winners/{str(year)}/{category}"
    res = httpx.get(url)
    if res.status_code == 200:
        return res.text
    return None

def process_actors_page(page: str):
    soup = BeautifulSoup(page, "html.parser")
    nominees = soup.select(".nom-bio-info-area")
    result = []
    for nom in nominees:
        nom_name = nom.find("h4").find("a").text
        role_as = nom.find("div", class_="role-as").text
        show_title = nom.find("div", class_="show-title").text
        winner = True if nom.find("div", class_="winner-accent-text").text.lower() == "winner" else False
        
        result.append({
            "name": nom_name,
            "role_as": role_as, 
            "show": show_title,
            "winner": winner
        })
    return result

def process_show_page(page: str):
    soup = BeautifulSoup(page, "html.parser")
    nominees = soup.select(".title")
    result = []
    for nom in nominees:
        nom_name = nom.find("h3").find("a").text
        winner = True if nom.find("div", class_="winner-accent-text").text.lower() == "winner" else False
        result.append({"name": nom_name, "winner": winner})
    return result

In [5]:
def get_actors(start: int, end: int):
    for year in range(end, start - 1, -1):
        data = []
        for category in acting_categories:
            page = get_emmys_page(year, category)
            res = process_actors_page(page)
            data.append({category: res})
        
        with open(f"./emmys/actors_{str(year)}.json", "w") as f:
            json.dump(data, f)

In [8]:
def get_shows(start: int, end: int):
    for year in range (end, start - 1, -1):
        data = []
        for category in show_categories:
            page = get_emmys_page(year, category)
            res = process_show_page(page)
            data.append({category: res})
         
        with open(f"./emmys/shows_{str(year)}.json", "w") as f:
                json.dump(data, f)

In [9]:
get_actors(2000, 2023)
get_shows(2000, 2023)

AttributeError: 'NoneType' object has no attribute 'text'